# 🌾 Chana Quality Classifier - RF-DETR Training (SOTA)

Train RF-DETR (State-of-the-Art) model for chana quality detection.

**RF-DETR**: First real-time model to achieve 60+ mAP on COCO!

**Dataset**: 35,019 images, 9 classes

**Runtime**: Select GPU (Runtime → Change runtime type → T4 GPU or A100 recommended)

## 1. Setup Environment

In [ ]:
# Check GPU
!nvidia-smi

In [ ]:
# Install RF-DETR
!pip install rfdetr -q
!pip install pycocotools -q
print("RF-DETR installed!")

## 2. Mount Google Drive & Load Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Unzip COCO format dataset from Google Drive
# Make sure you uploaded dataset_coco.zip to your Drive root
!unzip -q /content/drive/MyDrive/dataset_coco.zip -d /content/
print("Dataset extracted!")

In [ ]:
# Verify COCO format dataset structure
!ls /content/dataset_coco/
!ls /content/dataset_coco/train/
!echo "Train images:" && ls /content/dataset_coco/train/*.jpg 2>/dev/null | wc -l || find /content/dataset_coco/train -name '*.jpg' | wc -l

## 3. Convert YOLO to COCO Format (if needed)

Run this section ONLY if you uploaded YOLO format dataset instead of COCO format.

In [ ]:
# Skip if you already have COCO format!
# This converts YOLO format to COCO JSON format

import json
import os
from PIL import Image
from tqdm import tqdm

CLASS_NAMES = [
    "Bad Black Channa",
    "Bad Kabuli Chana",
    "Good Black Channa",
    "Good Kabuli Chana",
    "foreign material",
    "Bad Soya",
    "Good Soya",
    "Bad Matar",
    "Good Matar"
]

def yolo_to_coco(yolo_dir, output_dir, split):
    """Convert YOLO format to COCO JSON format."""
    images_dir = os.path.join(yolo_dir, split, 'images')
    labels_dir = os.path.join(yolo_dir, split, 'labels')
    
    os.makedirs(os.path.join(output_dir, split), exist_ok=True)
    
    coco = {
        "images": [],
        "annotations": [],
        "categories": [{"id": i, "name": name} for i, name in enumerate(CLASS_NAMES)]
    }
    
    ann_id = 1
    
    image_files = sorted([f for f in os.listdir(images_dir) if f.endswith('.jpg')])
    
    for img_id, img_file in enumerate(tqdm(image_files, desc=f"Converting {split}"), 1):
        img_path = os.path.join(images_dir, img_file)
        label_path = os.path.join(labels_dir, img_file.replace('.jpg', '.txt'))
        
        # Get image dimensions
        with Image.open(img_path) as img:
            width, height = img.size
        
        # Copy image to output
        import shutil
        shutil.copy(img_path, os.path.join(output_dir, split, img_file))
        
        coco["images"].append({
            "id": img_id,
            "file_name": img_file,
            "width": width,
            "height": height
        })
        
        # Parse YOLO labels
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) >= 5:
                        class_id = int(parts[0])
                        x_center, y_center, w, h = map(float, parts[1:5])
                        
                        # Convert to COCO format (absolute coordinates)
                        x = (x_center - w/2) * width
                        y = (y_center - h/2) * height
                        w = w * width
                        h = h * height
                        
                        coco["annotations"].append({
                            "id": ann_id,
                            "image_id": img_id,
                            "category_id": class_id,
                            "bbox": [x, y, w, h],
                            "area": w * h,
                            "iscrowd": 0
                        })
                        ann_id += 1
    
    # Save COCO JSON
    json_path = os.path.join(output_dir, split, '_annotations.coco.json')
    with open(json_path, 'w') as f:
        json.dump(coco, f)
    
    print(f"{split}: {len(coco['images'])} images, {len(coco['annotations'])} annotations")

# Convert all splits
YOLO_DIR = '/content/dataset'
COCO_DIR = '/content/dataset_coco'

for split in ['train', 'valid', 'test']:
    yolo_to_coco(YOLO_DIR, COCO_DIR, split)

print("\nConversion complete!")

## 4. Train RF-DETR

In [ ]:
from rfdetr import RFDETRBase
from rfdetr.util.coco_classes import COCO_CLASSES

# Initialize model
model = RFDETRBase()

# Train
model.train(
    dataset_dir="/content/dataset_coco",
    epochs=50,                    # Increase for better results
    batch_size=8,                 # Reduce if OOM (T4 = 8, A100 = 16)
    grad_accum_steps=4,           # Effective batch = 8*4 = 32
    lr=1e-4,
    output_dir="/content/drive/MyDrive/chana_models/rfdetr_v1",
    checkpoint_freq=10,
    use_ema=True,
    use_wandb=False,              # Set True for logging
)

## 5. Evaluate on Test Set

In [ ]:
# Load trained model
from rfdetr import RFDETRBase

model = RFDETRBase()
model.load("/content/drive/MyDrive/chana_models/rfdetr_v1/best.pt")

# Evaluate
results = model.evaluate(
    dataset_dir="/content/dataset_coco",
    split="test"
)

print(f"\nTest Results:")
print(f"mAP50: {results['map50']:.4f}")
print(f"mAP50-95: {results['map']:.4f}")

## 6. Test Inference

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
from PIL import Image
import random
import os

# Get random test image
test_dir = '/content/dataset_coco/test'
test_images = [f for f in os.listdir(test_dir) if f.endswith('.jpg')]
random_img = random.choice(test_images)
img_path = os.path.join(test_dir, random_img)

# Run inference
image = Image.open(img_path)
detections = model.predict(image, threshold=0.5)

# Draw results
import numpy as np
img = np.array(image)
img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

CLASS_NAMES = [
    "Bad Black Channa", "Bad Kabuli Chana", "Good Black Channa",
    "Good Kabuli Chana", "foreign material", "Bad Soya",
    "Good Soya", "Bad Matar", "Good Matar"
]

for det in detections:
    x1, y1, x2, y2 = map(int, det['bbox'])
    cls = det['class_id']
    conf = det['confidence']
    label = f"{CLASS_NAMES[cls]}: {conf:.2f}"
    
    color = (0, 255, 0) if 'Good' in CLASS_NAMES[cls] else (0, 0, 255)
    cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
    cv2.putText(img, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

cv2_imshow(img)

## 7. Export Model

In [ ]:
# Export to ONNX
model.export(
    format="onnx",
    output_path="/content/drive/MyDrive/chana_models/rfdetr_v1/model.onnx"
)
print("Model exported to ONNX!")

## 8. Download Trained Model

In [ ]:
from google.colab import files
files.download('/content/drive/MyDrive/chana_models/rfdetr_v1/best.pt')